In [1]:
import datetime as dt
import itertools
import inspect
import logging
import matplotlib.pyplot as plt
import operator
import os
import pandas as pd
import pickle
import random
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import json
from random import shuffle
from src.product import Product
from src.warehouse import Warehouse
from src.shelf import Shelf

## Data loading and preprocessing

In [ ]:
idc_1 = pd.read_csv('items.csv')
df_sales_period = pd.read_csv('orders.csv')
df_sales_1 = pd.read_csv('orders1day.csv')

## Create warehouse topology


In [ ]:
def create_warehouse(asile_lenght = 1, asile_size = 1, asile = 1):
    warehouse = Warehouse(20, 42)

    for row in range(2,18):
        for col in range(1, 41):
            if col % 3 == 1 or col % 3 == 2:
                warehouse.add_shelf(row, col, 5)
    return warehouse

In [ ]:
def get_product_list():
    product_list = list() 
    for index, row in idc_1.iterrows():
            curr_stock = 1
            curr_volume = row['Volume']
            for i in range(row['CurrentStock']):
                product = Product(id=row['ItemId'], volume_liters=row['Volume'], idc=curr_stock/row['AdsNum'], coi = (row['Volume']*row['CurrentStock'])/((row['AdsNum'] * row['StockAvailDays'])/28), coi_with_ads = (curr_volume)/(row['AdsNum']), exemplar_id = i)
                product_list.append(product)
                curr_stock += 1
                curr_volume += curr_volume
    return product_list

In [ ]:
def get_exemplars_order(products, item_id, already_taken_product, metric):
    filtered_products = [product for product in products if product.id == item_id and (product.id, product.exemplar_id) not in already_taken_product]
    
    if not filtered_products:
        return None
    if(metric == 'idc'):
        lowest_idc_product = filtered_products[0]
        for product in filtered_products:
            if product.idc < lowest_idc_product.idc:
                lowest_idc_product = product
        return lowest_idc_product    
    if(metric == 'coi'):
        lowest_coi_product = filtered_products[0]
        for product in filtered_products:
            if product.coi < lowest_coi_product.coi:
                lowest_coi_product = product
        return lowest_coi_product
    if(metric == 'random'):
        return random.choice(filtered_products)    

In [ ]:
product_list = get_product_list()

## Shared

In [ ]:
statistics = {}

In [ ]:
period_statistics = {}

## IDC s_shape

In [ ]:
warehouse = create_warehouse()

In [ ]:
product_list = sorted(product_list, key = lambda x: x.idc)


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
unique_orders = df_sales_1['OmsPostingId'].unique()

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для IDC")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега s-shape пробегов за период 30 дней размещения модели для IDC")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'IDC s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
order_items

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
warehouse.visualize_warehouse('occupancy')

## IDC return

In [ ]:
warehouse = create_warehouse()

In [ ]:
product_list = sorted(product_list, key = lambda x: x.idc)


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для IDC")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для IDC")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'IDC return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat) , 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
warehouse.visualize_warehouse('occupancy')

## COI s_shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
df_sales_1

In [ ]:
print("Статистика пробега s-shape пробегов размещения модели для COI")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date, 'COI s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

## COI return shape

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для COI")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            if(exmp is None):
                continue
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для COI")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['COI return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## IDC by exp

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi_with_ads)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 's_shape')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для IDC by exp")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для IDC by exp")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI with ads s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC by exp s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat) , 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## IDC by exp

In [ ]:
warehouse = create_warehouse()

product_list = sorted(product_list, key = lambda x: x.coi_with_ads)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для IDC by exp")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'coi')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для IDC by exp")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'COI with ads return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['IDC by exp return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Random return

In [ ]:
warehouse = create_warehouse()

shuffle(product_list)

for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'return')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для randoim")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'random')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            exemplars_in_order.append(exmp)
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("return", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега return пробегов за период 30 дней размещения модели для Random")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("return", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'random return')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['Random return'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Random s-shape

In [ ]:
warehouse = create_warehouse()


for product in product_list:
    nearest_empty_cell = warehouse.place_product(product, 'random')

warehouse.visualize_warehouse('occupancy')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('coi')

In [ ]:
warehouse.visualize_warehouse('idc')

In [ ]:
print("Статистика пробега return пробегов размещения модели для random")
stat = []
order_stat = []
close_to_drop_stat = []
already_picked_items = list()
for order_id in unique_orders:
    order_items = df_sales_1[df_sales_1['OmsPostingId'] == order_id]
    exemplars_in_order = list()
    locations = list()

    print(order_id)
    for _, item in order_items.iterrows():
        for i in range(item['Quantity']):
            exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'random')
            already_picked_items.append((exmp.id, exmp.exemplar_id))
            location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
            if(location is None):
                continue
            locations.append((location, exmp))
            close_to_drop_stat.append(abs(0 - location[0]) + abs(0 - location[1]))
            exemplars_in_order.append(exmp)
    if(len(locations) == 0):
        continue
    print(already_picked_items)
    dist = warehouse.perform_order_pick("s_shape", locations, True)
    stat.append(dist)
    order_stat.append(len(locations))

In [ ]:
unique_dates = df_sales_period['Date'].unique()

print("Статистика пробега s_shape пробегов за период 30 дней размещения модели для Random")

for date in unique_dates:
    stat_period = []
    order_stat_period = []
    close_to_drop_stat_period = []
    already_picked_items_period = list()
    orders_for_date = df_sales_period[df_sales_period['Date'] == date]
    unique_orders_for_date = orders_for_date['OmsPostingId'].unique()
    for order_id in unique_orders_for_date:
        order_items = orders_for_date[orders_for_date['OmsPostingId'] == order_id]
        exemplars_in_order = list()
        locations = list()

        print(order_id)
        for _, item in order_items.iterrows():
            for i in range(item['Quantity']):
                exmp = get_exemplars_order(product_list, item['RezonItemId'], already_picked_items, 'idc')
                if(exmp is None):
                    continue        
                already_picked_items_period.append((exmp.id, exmp.exemplar_id))
                location = warehouse.find_product_location(exmp.id, exmp.exemplar_id)
                if(location is None):
                    continue
                locations.append((location, exmp))
                close_to_drop_stat_period.append(abs(0 - location[0]) + abs(0 - location[1]))
                exemplars_in_order.append(exmp)
        if(len(locations) == 0):
            continue
        print(already_picked_items)
        dist = warehouse.perform_order_pick("s_shape", locations, True)
        stat_period.append(dist)
        order_stat_period.append(len(locations))
    period_statistics[(date,'random s_shape')] = {'mean' : np.mean(stat_period), 
                                                'median': np.median(sorted(stat_period)), 
                                                'sum': sum(stat_period), 
                                                'avg num of items': np.mean(order_stat_period), 
                                                'order_dist' : stat_period, 
                                                'mean dist to drop': np.mean(close_to_drop_stat_period), 
                                                'median dist to drop': np.median(close_to_drop_stat_period), 
                                                'dist to dtop': close_to_drop_stat_period}

In [ ]:
unique_dates

In [ ]:
np.mean(stat)

In [ ]:
np.median(sorted(stat))

In [ ]:
sum(stat)

In [ ]:
statistics['random s-shape'] = {'mean' : np.mean(stat), 'median': np.median(sorted(stat)), 'sum': sum(stat), 'avg num of items': np.mean(order_stat), 'order_dist' : stat, 'mean dist to drop': np.mean(close_to_drop_stat), 'median dist to drop': np.median(close_to_drop_stat), 'dist to dtop': close_to_drop_stat}

## Statistics

In [ ]:
for metric, stats in statistics.items():

    print(f"Метрика: {metric}")
    print("Статистики:")
    for stat_name, value in stats.items():
        if(metric == 'order_dist'):
            continue
        print(f"{stat_name}: {value}")
    print()

In [ ]:
s_shape_metrics = {key: value for key, value in statistics.items() if 's-shape' in key}

# Построить графики для метрик s-shape
plt.figure(figsize=(50, 9))
for key, value in s_shape_metrics.items():
    order_dist = value['order_dist']
    num_items = len(order_dist)
    indices = np.arange(1, num_items + 1)
    plt.plot(indices, order_dist,  linestyle='-', linewidth = 4, label=key)

plt.xlabel('Index')
plt.ylabel('Order Distance')
plt.title(' ')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
s_shape_metrics = {key: value for key, value in statistics.items() if 'return' in key}

# Построить графики для метрик s-shape
plt.figure(figsize=(50, 9))
for key, value in s_shape_metrics.items():
    order_dist = value['order_dist']
    num_items = len(order_dist)
    indices = np.arange(1, num_items + 1)
    plt.plot(indices, order_dist,  linestyle='-', linewidth = 4, label=key)

plt.xlabel('Index')
plt.ylabel('Order Distance')
plt.title('Количество пробегов для одного и того же заказа с эвристикой "return" в разрезе статистик')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
len(period_statistics)

In [ ]:
# Перебор данных в period_statistics для каждой даты
for (date, metric), stats_df in period_statistics.items():
    print(f"Дата: {date}")
    print(f"Метрика: {metric}")
    print("Статистики:")
    # Перебор метрик для данной даты
    for stat_name, value in stats_df.items():
        if metric == 'order_dist':
            continue
        print(f"{stat_name}: {value}")



In [ ]:
len(period_statistics)

In [ ]:
# Создаем словарь для хранения средних значений для метрик, содержащих 's_shape' в названии
mean_s_shape_metrics = {metric: [] for (date, metric) in period_statistics.keys() if 's_shape' in metric}
unique_dates = set()

# Перебираем данные в period_statistics
for (date, metric), stats_df in period_statistics.items():
    if 's_shape' in metric:
        unique_dates.add(date)
        mean_value = stats_df['mean']  # Предполагается, что 'mean' - это название столбца со средними значениями
        mean_s_shape_metrics[metric].append(mean_value)

dates = sorted(list(unique_dates))
# Строим график для каждой метрики, содержащей 's_shape' в названии
plt.figure(figsize=(10, 6))
for metric, values in mean_s_shape_metrics.items():
    if values:  # Проверяем, есть ли значения для данной метрики
        plt.plot(dates, values, label=metric)

plt.xlabel('Дата')
plt.ylabel('Среднее значение')
plt.title('Средние значения метрик с s_shape по датам')
plt.xticks(rotation=45)  # Поворачиваем подписи по оси X для лучшей читаемости
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

